<a href="https://colab.research.google.com/github/javierbz2000/Practica-final/blob/main/Practica_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PRACTICA FINAL**

Realizada por: 
- Javier Bañares
- Manuel Ramirez
- Javier Amezaga
- Claudia Corbella

# Importacion de librerias

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from pandas.core.algorithms import mode
from sklearn.inspection import permutation_importance
import multiprocessing
from time import sleep
from random import shuffle
from numpy import atleast_2d
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KDTree
#from hdbscan import HDBSCAN
from matplotlib.colors import ListedColormap
from matplotlib import cm
import matplotlib.ticker as ticker
import plotly.io as plot_io
import plotly.graph_objects as go
from numpy.random import rand, seed
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.neighbors import kneighbors_graph
from sklearn.datasets import make_blobs, make_circles, make_moons,load_iris
from matplotlib.colors import ListedColormap
from sklearn.cluster import KMeans, SpectralClustering, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.manifold import TSNE, Isomap
import seaborn as sns
import plotly.express as px
from sklearn.impute import KNNImputer
import scipy.stats as ss
import warnings
import sklearn
from sklearn.pipeline import Pipeline
!pip install category_encoders
import category_encoders as ce
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, GradientBoostingRegressor

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve, accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix, plot_confusion_matrix, adjusted_mutual_info_score, \
                            mean_squared_error

from sklearn.model_selection import cross_val_score,train_test_split, RepeatedKFold, KFold, GridSearchCV, ParameterGrid



# Importacion de la base de datos

In [ ]:
Datos = pd.read_csv("NCDB_1999_to_2014.csv")
Datos.head()

# Analisis exploratorio inicial

In [ ]:
Datos.shape

Un primer analisis exploratorio muestra que el dataset tiene inicialmente un total de 5860405 filas y 22 columnas

## Tipos de datos

In [ ]:
Datos.dtypes

Podemos observar que la mayoria de las variables son tipo object a pesar de que la mayoria contienen datos numericos. Estos es porque en muchas hay valores no numericos cuyo significado equivalen a valores NA y estos son valores UU (Unknown), QQ o NN.

Mas adelante realizaremos un analisis sobre estos valores, y segun los resultados, decidiremos como tratarlos. 


In [ ]:
Datos.info()

In [ ]:
Datos.select_dtypes(include='object').nunique()

Con la función unique podemos observar los diferentes valores/opciones que podemos encontrar dentro de cada variable

In [ ]:
Datos.select_dtypes(include='int64').nunique()

Observamos que la severidad del accidente cuenta con dos valores, 1 si la colisión cuenta con una fatalidad y 2 si la colisión no hay ninguna fatalidad, por lo que la usaremos como variable de referencia a la hora de tratar la acción correctiva de la prima y el descuento de la prima

In [ ]:
#corr_datos = Datos.corr(method='pearson')

#plt.figure(figsize=(8, 6))
#sns.heatmap(corr_datos, annot=True)
#plt.show()

# Tratamiento de valores faltantes (MISSING, UNKNOWN...)

Comenzamos realizando un analisis de los valores missing sabiendo cuantos hay en cada una de las variables 

In [ ]:
missing_values_count = Datos.isna().sum()
missing_values_count

El numero de missings es muy bajo por lo que decidimos eliminarlos, ya que no va a suponer una distorsion de los datos 

In [ ]:
Datos=Datos.dropna()

A través de los siguientes codigos hemos comenzado analizando la cantidad de valores de tipo U, N, X y Q que hay por filas, y segun los resultados decimos que hacer PARA su tratamiento 

In [ ]:
pd_series_U_filas = Datos.isin(["U", "UU", "UUUU"]).sum(axis=1).sort_values(ascending=False)
total_U_f= pd.DataFrame(pd_series_U_filas, columns=['U_filas'])   
total_U_f['porcentaje_filas_U'] = (total_U_f['U_filas']/Datos.shape[1])*100
total_U_f


In [ ]:
pd_series_N_filas = Datos.isin(["N","NN","NNNN"]).sum(axis=1).sort_values(ascending=False)
total_N_f= pd.DataFrame(pd_series_N_filas, columns=['N_filas'])   
total_N_f['porcentaje_filas_N'] = (total_N_f['N_filas']/Datos.shape[1])*100
total_N_f

In [ ]:
pd_series_Q_filas = Datos.isin(["Q", "QQ", "QQQQ"]).sum(axis=1).sort_values(ascending=False)
total_Q_f= pd.DataFrame(pd_series_Q_filas, columns=['Q_filas'])   
total_Q_f['porcentaje_filas_Q'] = (total_Q_f['Q_filas']/Datos.shape[1])*100
total_Q_f

Una de las opciones que tenemos es eliminar todos los valores, pero al haber una gran cantidad disttorsionaria los datos. Por lo tanto hemos decidido eliminar todas las filas que contengan mas de un 30% de cada tipo de estos valores. Esto lo hemos hecho utilizando el threshold

In [ ]:
threshold=30
list_vars_not_U=total_U_f[total_U_f['porcentaje_filas_U']>threshold]
Columnas_drop_u= list(list_vars_not_U.index)
Datos= Datos.drop(Columnas_drop_u)
Datos.shape

In [ ]:
threshold=30
list_vars_not_N=total_U_f[total_N_f['porcentaje_filas_N']>threshold]
Columnas_drop_N= list(list_vars_not_N.index)
Datos= Datos.drop(Columnas_drop_N)
Datos.shape

In [ ]:
threshold=30
list_vars_not_Q=total_Q_f[total_Q_f['porcentaje_filas_Q']>threshold]
Columnas_drop_Q= list(list_vars_not_Q.index)
Datos= Datos.drop(Columnas_drop_Q)
Datos.shape

Una vez hecho esto, hemos analizado el porcentaje de cada uno de los valores U, N y Q que tiene cada variable por columna. 

In [ ]:
pd_series_U_columns = Datos.isin(["U", "UU", "UUUU"]).sum().sort_values(ascending=False)
total_U_c= pd.DataFrame(pd_series_U_columns, columns=['U_columnas'])   
total_U_c['porcentaje_columnas'] = (total_U_c['U_columnas']/Datos.shape[0])*100
total_U_c

In [ ]:
pd_series_N_columns = Datos.isin(["N","NN","NNNN"]).sum().sort_values(ascending=False)
total_N_c= pd.DataFrame(pd_series_N_columns, columns=['N_columnas'])   
total_N_c['porcentaje_columnas_N'] = (total_N_c['N_columnas']/Datos.shape[0])*100
total_N_c

In [ ]:
pd_series_Q_columns = Datos.isin(["Q", "QQ", "QQQQ"]).sum().sort_values(ascending=False)
total_Q_c= pd.DataFrame(pd_series_Q_columns, columns=['Q_columnas'])   
total_Q_c['porcentaje_columnas_Q'] = (total_Q_c['Q_columnas']/Datos.shape[0])*100
total_Q_c

Vemos que el porcentaje en los tres tipos esta por debajo del 30%, pero aun sigue habiendo. El % de valores N y Q es bastante bajo por lo que vamos a eliminar dichas filas. 

## ELIMINACION DE TODOS LOS VALORES "N" Y "Q"

In [ ]:
Datos = Datos.drop(Datos[Datos['P_SAFE']=="NN"].index)
Datos = Datos.drop(Datos[Datos['P_ISEV']=="N"].index)
Datos = Datos.drop(Datos[Datos['V_YEAR']=="NNNN"].index)
Datos = Datos.drop(Datos[Datos['V_TYPE']=="NN"].index)
Datos = Datos.drop(Datos[Datos['P_AGE']=="NN"].index)
Datos = Datos.drop(Datos[Datos['P_SEX']=="N"].index)
Datos = Datos.drop(Datos[Datos['P_PSN']=="NN"].index)
Datos = Datos.drop(Datos[Datos['C_CONF']=="QQ"].index)
Datos = Datos.drop(Datos[Datos['C_RSUR']=="Q"].index)
Datos = Datos.drop(Datos[Datos['C_RCFG']=="QQ"].index)
Datos = Datos.drop(Datos[Datos['C_TRAF']=="QQ"].index)
Datos = Datos.drop(Datos[Datos['C_RALN']=="Q"].index)
Datos = Datos.drop(Datos[Datos['P_PSN']=="QQ"].index)
Datos = Datos.drop(Datos[Datos['C_WTHR']=="Q"].index)
Datos = Datos.drop(Datos[Datos['P_SAFE']=="QQ"].index)
Datos = Datos.drop(Datos[Datos['V_TYPE']=="QQ"].index)

## SUSTITUCION DE TODOS LOS VALORES UNKNOWN POR LA MODA

Sin embargo, como se observa anteriormente, la cantidad de U sigue siendo alta en la mayoria de los datos por lo que hemos optado por sustituir estos valores por la moda de cada columna.

In [ ]:
UU_columns=Datos.isin(["U", "UU", "UUUU"]).columns.tolist()
def mode_columns (df):
  for column in df:
    if(column in UU_columns):
      df[column]=df[column].replace("U", df[column].mode()[0])
      df[column]=df[column].replace("UU", df[column].mode()[0])
      df[column]=df[column].replace("UUUU", df[column].mode()[0])
   

  return mode_columns

In [ ]:
mode_columns(Datos)

La variable P_AGE no se ve afectada y sigue con datos desconocidos, por lo que puede deberse a que la moda sean esos datos desconocidos, por tanto se procederá a tratar así.

In [ ]:
Datos = Datos.drop(Datos[Datos['P_AGE']=="UU"].index)

Tras estas modificaciones la tabla resultante pasa a tener XXX filas y mantienen las mismas columnas

In [ ]:
Datos.shape

Estas modificaciones nos permiten tratar las variables confundidas y convertirlas en tipo int (numerico)

In [ ]:
Datos.dtypes

In [ ]:
Datos['C_MNTH'] = Datos['C_MNTH'].astype(int)
Datos['C_WDAY'] = Datos['C_WDAY'].astype(int)
Datos['C_HOUR'] = Datos['C_HOUR'].astype(int)
Datos['C_VEHS'] = Datos['C_VEHS'].astype(int)
Datos['C_CONF'] = Datos['C_CONF'].astype(int)
Datos['C_WTHR'] = Datos['C_WTHR'].astype(int)
Datos['C_RSUR'] = Datos['C_RSUR'].astype(int)
Datos['C_RALN'] = Datos['C_RALN'].astype(int)
Datos['C_TRAF'] = Datos['C_TRAF'].astype(int)
Datos['C_RCFG'] = Datos['C_RCFG'].astype(int)
Datos['V_ID'] = Datos['V_ID'].astype(int)
Datos['V_TYPE'] = Datos['V_TYPE'].astype(int)
Datos['V_YEAR'] = Datos['V_YEAR'].astype(int)
Datos['P_ID'] = Datos['P_ID'].astype(int)
Datos['P_PSN'] = Datos['P_PSN'].astype(int)
Datos['P_ISEV'] = Datos['P_ISEV'].astype(int)
Datos['P_USER'] = Datos['P_USER'].astype(int)
Datos['P_SAFE'] = Datos['P_SAFE'].astype(int)
Datos['P_AGE'] = Datos['P_AGE'].astype(int)


Tras estas modificaciones, la unica variable tipo object que queda es la variable P_SEX que procederemos mas adelante a convertirla en una variable tipo dummy para poder hacer el analisis y tener toda la tabla con datos tipo numerico.

# Separacion del modelo en train y test

In [ ]:
from sklearn.model_selection import train_test_split
X_Datos_train, X_Datos_test, y_Datos_train, y_Datos_test = train_test_split(Datos.drop("C_SEV",axis=1), 
                                                                     Datos["C_SEV"], 
                                                                     stratify= Datos["C_SEV"], 
                                                                     test_size=0.2)
Datos_train = pd.concat([X_Datos_train, y_Datos_train],axis=1)
Datos_test = pd.concat([X_Datos_test, y_Datos_test],axis=1)

Con la funcion train_test_split separamos las tabla en un grupo train y otro test. Para dicha separacion hemos tenido en cuenta el balanceo de datos de la variable principal

Los datos del grupo train estan en la variable "Datos_train" y los datos del grupo test en la variable "Datos_test". 

# Transforacion variables categoricas

Una vez divido el data set en train y test, transformamos en el Data set de datos train que esta en la variable "Datos_train", la unica variable tipo object que nos queda "P_SEX" utilizando la fundion OneHotEncoder. Incluimos el dataset resultante en la variable Datos_train_dummy




In [ ]:
Datos_train=Datos_train.set_index(["C_SEV"])
list_columns_cat = list(Datos_train.select_dtypes("object").columns)
list_other = list(set(Datos_train.columns)-set(list_columns_cat))

In [ ]:
ohe = ce.OneHotEncoder(cols=list_columns_cat)
model = ohe.fit(Datos_train)

In [ ]:
Datos_train_dummy = model.transform(Datos_train)
Datos_train_dummy.dtypes.to_dict()

Relizamos el mismo proceso de conversion de la variable P_SEX para el dataset test que esta en la variable Datos_test. Incluimos el dataset resultante en la varible Datos_test_dummy


In [ ]:
list_columns_cat_test = list(Datos_test.select_dtypes("object").columns)
list_other_test = list(set(Datos_test.columns)-set(list_columns_cat_test))

In [ ]:
ohe = ce.OneHotEncoder(cols=list_columns_cat_test)
model = ohe.fit(Datos_test)

In [ ]:
Datos_test_dummy = model.transform(Datos_test)
Datos_test_dummy.dtypes.to_dict()

# ¿Qué tipos de vehículos (modelos, antigüedad, etc.) y conductores son más propensos a tener accidentes (acción correctiva en prima)?

Para analizar cuales son las caracteristicas de los vehiculos y las caracteristicas de los conductores que tienen una mayor propension a tener mas accidentes, hemos considerado que vamos relizar un analisis utilizando la funcion groupby()
Comenzamos creando una nueva variable en la que solo se encuentran las variables que nos pueden proporcionar estas caracteristicas, a partir del data set que se encuentra en Datos_test_dummy y Datos_train_dummy . Estas variables son: V_YEAR, V_TYPE,P_AGE, P_SEX_1, P_SEX_2, P_USER, P_SAFE, P_PSN y P_ISEV

Por lo tanto vamos a utilizar unicamente las variables X_Datos_train_acc y X_Datos_test_acc que contienen unicamente las variables mencionadas anteriormente a partir de los datos train y test que tiene las variables tipo object ya transformadas.

In [ ]:
X_Datos_train_acc=Datos_train_dummy[["V_YEAR","V_TYPE","P_AGE","P_SEX_1","P_USER","P_SAFE", "P_PSN", "P_ISEV"]]
X_Datos_test_acc=Datos_test_dummy[["V_YEAR","V_TYPE","P_AGE","P_SEX_1","P_USER","P_SAFE","P_PSN", "P_ISEV","C_SEV"]]
X_Datos_train_acc.head()	

La columna V_YEAR contiene los datos del año en el que el vehiculo fue producido. La fecha minima de esta columna es 1901 y la fecha maxima el 2015. Teniendo en cuenta que son 114 años entre años hemos optado por analizarlo de dos formas. Por una parte, agrupar los datos por las fechas que contiene la columna "V_YEAR" y por otra, para facilitar el analisis y obtener resultados mas generales, hemos creado una nueva columna que contiene los terciles de la columna V_YEAR y segun el año del vehiculo, obtendra una etiqueta u otra. Esta nueva columna es Year_range


In [ ]:
conditionlist = [
    (X_Datos_train_acc["V_YEAR"]>=1901) & (X_Datos_train_acc["V_YEAR"]<=1939) ,
    (X_Datos_train_acc["V_YEAR"]>=1940) & (X_Datos_train_acc["V_YEAR"]<=1978),
    (X_Datos_train_acc["V_YEAR"]>=1979) & (X_Datos_train_acc["V_YEAR"]<=2015)]

choicelist = ['1901-1939', '1940-1978',"1979-2015"]
X_Datos_train_acc["Year_range"] = np.select(conditionlist, choicelist,default='otro')
X_Datos_train_acc

Como ocurre con la variable V_YEAR, la variable P_AGE contine la edad del conductor, al ser el numero maximo 99 y el minimo 1 hay muchos valores por lo que vamos a crear una nueva columna en la que dividimos dicha variable en tramos.Esta nueva columna se llama AGE_RANGE y contiene un total de 10 rangos el primero va de 1 a 18 y el segundo de 18 a 20, a apartir del tercero van de 10 en 10.


In [ ]:
conditionlistt = [
    (X_Datos_train_acc["P_AGE"]>=1) & (X_Datos_train_acc["P_AGE"]<=18) ,
    (X_Datos_train_acc["P_AGE"]>=18) & (X_Datos_train_acc["P_AGE"]<=20),
    (X_Datos_train_acc["P_AGE"]>=21) & (X_Datos_train_acc["P_AGE"]<=30),
    (X_Datos_train_acc["P_AGE"]>=31) & (X_Datos_train_acc["P_AGE"]<=40),
    (X_Datos_train_acc["P_AGE"]>=41) & (X_Datos_train_acc["P_AGE"]<=50),
    (X_Datos_train_acc["P_AGE"]>=51) & (X_Datos_train_acc["P_AGE"]<=60),
    (X_Datos_train_acc["P_AGE"]>=61) & (X_Datos_train_acc["P_AGE"]<=70),
    (X_Datos_train_acc["P_AGE"]>=71) & (X_Datos_train_acc["P_AGE"]<=80),
    (X_Datos_train_acc["P_AGE"]>=81) & (X_Datos_train_acc["P_AGE"]<=90),
    (X_Datos_train_acc["P_AGE"]>=91) & (X_Datos_train_acc["P_AGE"]<=99)]

choicelistt = ['1-18', '18-20',"20-30","30-40","40-50","50-60","60-70","70-80","80-90","90-99"]
X_Datos_train_acc["AGE_RANGE"] = np.select(conditionlistt, choicelistt)
X_Datos_train_acc


Una vez creada la nueva columna, procedemos a resetear el index el cual es la columna C_SEV 

In [ ]:
X_Datos_train_acc=X_Datos_train_acc.reset_index()

Asi mismo, como menciona el apartado, unicamente queremos saber las caracteristicas de los condcutores. Esta informacion la contiene la el valor 11 de la variable P_PSN o el valor 1 d ela variable P_USER. Podemos seleccionar el valor de una o de otra indiferentemente. Por lo que unicamente vamos a selecionar ese valor 11 de la variable P_PSN


In [ ]:
X_Datos_train_acc= X_Datos_train_acc[X_Datos_train_acc["P_PSN"].isin([11])]

Como hemos mencionando anteriormente, la variable C_SEV contiene dos valores:

    1-> cuando la colision ha producido al menos una fatalidad
    2-> cuando la colision no ha producido ninguna fatalidad

En base a esto, como no hay ninguna columna que contenga los datos sobre la accion correctiva en prima o el descuenta en prima, hemos interpretado que para establecer una u otra va a depender de si hay al menos una fatalidad o de si no hay ninguna fatalidad.

Para el caso de la accion correctiva en prima esta se aplicara su la colision cuenta con al menos 1 fatalidad. Esta informacion la contiene el valor "1" de la variable C_SEV y como es lo que queremos estudiar en ete apartado, procedemos a selecionar unicamente los valores "1" de esta columna. 


In [ ]:

X_Datos_train_acc1= X_Datos_train_acc[X_Datos_train_acc["C_SEV"].isin([1])]

Agrupamos la variable V_TYPE y sumamos cuantos datos C_SEV hay en cada valor de variable por la que agrupamos

In [ ]:

V_TYPE1 = X_Datos_train_acc1[["C_SEV","V_TYPE"]].groupby(["V_TYPE"]).count()
V_TYPE1


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(10,5))
V_TYPE1.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones con fatalidad por tipo de vehiculo')

Como comentabamos previamente. Para el analisis del año de produccion de cada vehiculo, primero agrupamos la variable V_YEAR y sumamos cuantos datos C_SEV hay en cada valor de la variable por la que agrupamos


In [ ]:

V_YEAR1= X_Datos_train_acc1[['C_SEV','V_YEAR']].groupby(['V_YEAR']).count()
V_YEAR1


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(20,10))
V_YEAR1.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones con fatalidad por año del modelo de vehiculo')

En segundo lugar, para obtener unos resultados mas generales agrupamos la variable Year_range y sumamos cuantos datos C_SEV hay en cada valor por el que agrupamos 


In [ ]:


Year_range1= X_Datos_train_acc1[['C_SEV','Year_range']].groupby(['Year_range']).count()
Year_range1


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(15,8))
Year_range1.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones con fatalidad por intervalo de año del modelo de vehiculo')

Hemos dedicido que para el analisis de la edad unicamente vamos a realizar la agrupacion sobre la variable "AGE_RANGE"

In [ ]:

AGE_RANGE1=X_Datos_train_acc1[['C_SEV','AGE_RANGE']].groupby(['AGE_RANGE']).count()
AGE_RANGE1


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(15,8))
AGE_RANGE1.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones con fatalidad por intervalo de edad del conductor')

Agrupamos la variable P_SEX_1 que contiene un 1 si es mujer (F) y un 0 si  es hombre, y sumamos cuantos datos C_SEV hay en cada valor de variable por la que agrupamos


In [ ]:
P_SEX_1_1= X_Datos_train_acc1[['C_SEV','P_SEX_1']].groupby(['P_SEX_1']).count()
P_SEX_1_1


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(15,8))
P_SEX_1_1.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones con fatalidad por genero del conductor')

Agrupamos la variable P_USER y sumamos cuantos datos C_SEV hay en cada valor de variable por la que agrupamos

In [ ]:
P_USER1=X_Datos_train_acc1[['C_SEV','P_USER']].groupby(['P_USER']).count()
P_USER1


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(15,8))
P_USER1.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones con fatalidad por tipo de usuario')


Agrupamos la variable P_SAFE y sumamos cuantos datos C_SEV hay en cada valor de variable por la que agrupamos

In [ ]:

P_SAFE1=X_Datos_train_acc1[['C_SEV','P_SAFE']].groupby(['P_SAFE']).count()
P_SAFE1


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(15,8))
P_SAFE1.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones con fatalidad por tipo de seguridad')

Agrupamos la variable P_ISEV y sumamos cuantos datos C_SEV hay en cada valor de variable por la que agrupamos

In [ ]:
P_ISEV1=X_Datos_train_acc1[['C_SEV',"P_ISEV"]].groupby(["P_ISEV"]).count()
P_ISEV1


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(15,8))
P_ISEV1.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones con fatalidad por grado de severidad')

## Conclusiones


Entre las características de los vehículos que tienden a tener más accidente priman

-	Los vehículos tipo 1 que corresponden a los vehículos de tipo ligero (vehículo de pasajeros, furgonetas...)
-	La mayoría de las colisiones con fatalidad son de los vehículos cuyo modelo es del 1979- 2015, especialmente lo del año 2001


Por otro lado, entre las características de los conductores que tienden a tener más accidentes priman:

-	Los conductores entre las de edades de 20 y 60 años, con un repunte de los conductores cuyas edades oscilan entre los 20 y 30 años
-	Destacan más los accidentes con fatalidad cuyos conductores son hombres
-	Entre los tipos de usuarios que conducen en el momento del accidente, se encuentran los conductores de coches, los ciclistas y los motoristas
-	La mayoría llevan un elemento de seguridad, como el cinturón, en el comento de la colisión 
-	Sobre la severidad de la colisión destaca que gran parte mueren en el acto (3- fatalidad)

# ¿Qué tipos de vehículos (modelos, antigüedad, etc.) y conductores son menos propensos a tener accidentes (descuento en prima)?

Para analizar cuales son las caracteristicas de los vehiculos y las caracteristicas de los conductores que tienen una memor propension a tener accidentes, hemos considerado que vamos utilizar el metodo utilizado previamente con la funcion groupby()
Como hemos mencionando anteriormente, la variable C_SEV contiene dos valores:

    1-> cuando la colision ha producido al menos una fatalidad
    2-> cuando la colision no ha producido ninguna fatalidad

Como no hay ninguna columna que contenga los datos sobre el descuenta en prima, hemos interpretado que para establecer dicho descuento va a depender de si no hay ninguna fatalidad, cuya informacion la contiene el valor "2" de la variable C_SEV y como es lo que queremos estudiar en ete apartado, procedemos a selecionar a continuacion unicamente los valores "2" de esta columna. 

Para ello, vamos a utilizar el dataset train que se encuentra en la variable X_Datos_train_acc que ya tiene las columnas con la informacion necesaria para determinar las caracteristicas, la filtracion por los datos del conductor unicamente, el index reseteado, la columna Year_range y la columna AGE_RANGE creadas.

La nueva variable que contiene unicamente los valores "2" de la columna C_SEV es X_Datos_train_acc2

In [ ]:
X_Datos_train_acc2= X_Datos_train_acc[X_Datos_train_acc["C_SEV"].isin([2])]
X_Datos_train_acc2= X_Datos_train_acc[X_Datos_train_acc["C_SEV"].isin([2])]

Agrupamos la variable V_TYPE y sumamos cuantos datos C_SEV hay en cada valor de variable por la que agrupamos

In [ ]:

V_TYPE2= X_Datos_train_acc2[['C_SEV','V_TYPE']].groupby(["V_TYPE"]).count()
V_TYPE2


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(15,8))
V_TYPE2.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones sin fatalidad por tipo de vehiculo')

Como comentabamos previamente. Para el analisis del año de produccion de cada vehiculo, primero agrupamos la variable V_YEAR y sumamos cuantos datos C_SEV hay en cada valor de la variable por la que agrupamos


In [ ]:
V_YEAR2= X_Datos_train_acc2[['C_SEV','V_YEAR']].groupby(['V_YEAR']).count()
V_YEAR2


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(20,10))
V_YEAR2.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones sin fatalidad por año del modelo del vehiculo')

En segundo lugar, para obtener unos resultados mas generales agrupamos la variable Year_range y sumamos cuantos datos C_SEV hay en cada valor por el que agrupamos


In [ ]:
Year_range2= X_Datos_train_acc[['C_SEV','Year_range']].groupby(['Year_range']).count()
Year_range2


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(15,8))
Year_range2.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones sin fatalidad por rango de año del modelo del vehiculo')

In [ ]:
P_AGE2= X_Datos_train_acc2[['C_SEV','P_AGE']].groupby(['P_AGE']).count()
P_AGE2


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(20,10))
P_AGE2.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones sin fatalidad por rango de edad del conductor')

In [ ]:

AGE_RANGE2=X_Datos_train_acc2[['C_SEV','AGE_RANGE']].groupby(['AGE_RANGE']).count()
AGE_RANGE2
fig, ax2 = plt.subplots(nrows = 1, figsize=(15,8))


In [ ]:
AGE_RANGE2.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones con fatalidad por intervalo de edad del conductor')

Agrupamos la variable P_SEX_1 que contiene un 1 si es mujer (F) y un 0 si es hombre, y sumamos cuantos datos C_SEV hay en cada valor de variable por la que agrupamos


In [ ]:

P_SEX_1_2= X_Datos_train_acc2[['C_SEV','P_SEX_1']].groupby(['P_SEX_1']).count()
P_SEX_1_2


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(15,8))
P_SEX_1_2.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones sin fatalidad por genero del conductor')

Agrupamos la variable P_USER y sumamos cuantos datos C_SEV hay en cada valor de variable por la que agrupamos

In [ ]:

P_USER2=X_Datos_train_acc2[['C_SEV','P_USER']].groupby(['P_USER']).count()
P_USER2


In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(15,8))
P_USER2.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones sin fatalidad por tipo de usuario ')

Agrupamos la variable P_ISEV y sumamos cuantos datos C_SEV hay en cada valor de variable por la que agrupamos


In [ ]:
P_SAFE2=X_Datos_train_acc2[['C_SEV','P_SAFE']].groupby(['P_SAFE']).count()
fig, ax2 = plt.subplots(nrows = 1, figsize=(15,8))
P_SAFE2.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones sin fatalidad por la seguridad del conductor')

Agrupamos la variable P_ISEV y sumamos cuantos datos C_SEV hay en cada valor de variable por la que agrupamos


In [ ]:
P_ISEV2=X_Datos_train_acc1[['C_SEV',"P_ISEV"]].groupby(["P_ISEV"]).count()
P_ISEV2

In [ ]:
fig, ax2 = plt.subplots(nrows = 1, figsize=(15,8))
P_ISEV2.plot(ax = ax2, kind = 'bar')
ax2.title.set_text('Colisiones sin fatalidad por severidad del conductor')

## Conclusiones

Entre las características de los vehículos que tienden a tener menos accidente priman:

-	Los vehículos tipo 1 que corresponden a los vehículos de tipo ligero (vehículo de pasajeros, furgonetas...)
-	La mayoría de las colisiones con fatalidad son de los vehículos cuyo modelo es del 1979- 2015, especialmente los del año 2000


Por otro lado, entre las características de los conductores que tienden a tener menos accidentes priman:

-	Los conductores entre las de edades de 20 y 60 años, con un repunte de los conductores cuyas edades oscilan entre los 20 y 30 años
-	Destacan más los accidentes con fatalidad cuyos conductores son hombres
-	Entre los tipos de usuarios que conducen en el momento del accidente, se encuentran los conductores de coches, los ciclistas y los motoristas
-	La mayoría llevan un elemento de seguridad, como el cinturón, en el comento de la colisión 
-	Sobre la severidad de la colisión destaca que gran parte mueren en el acto (3- fatalidad)

# ¿Qué es lo que mas contribuye a que existan fallecimientos en un accidente?

Para este apartado, voy a tomar la variable C_SEV como objetivo, siendo 1 si hay fallecimientos en el accidente, y 2 si no ha habido fallecimientos. 

In [ ]:
porcentaje_f = Datos["C_SEV"].value_counts(normalize=True).mul(100).rename('percent').reset_index()
count_f = Datos["C_SEV"].value_counts().reset_index()
Union_f= pd.merge(porcentaje_f, count_f, on=['index'], how='inner')
fig = px.histogram(Union_f, x="index", y=['percent'], labels={'index':'fatality'})
fig.show()

Podemos ver como de desbalanceado está el dataset, solo un 1.7% de los accidentes suponen al menos un fallecimiento.

## Estacionalidad

In [ ]:
Datos_temporal = pd.DataFrame()
Datos_temporal["date"] = pd.DatetimeIndex(Datos["C_YEAR"].map(str) + "-" + Datos["C_MNTH"].map(str))
Datos_temporal["fatal"] = np.where(Datos["C_SEV"]==1, 1, 0)
Datos_temporal["non_fatal"] = np.where(Datos["C_SEV"]==2, 1, 0)

In [ ]:
tiempo_colisión = Datos_temporal.groupby('date')["fatal","non_fatal"].sum()
plot = tiempo_colisión.plot(figsize = (15,5), title = "Deadly colissions over time")
plot.set_xlabel("Year")
plot.set_ylabel("Number of deadly colissions")

Puedo observar que existe un ciclo en los accidentes no fatales, hay momentos del año en los que el numero de accidentes no fatales es mayor.

Ahora hago la misma grafica para ver solo los accidentes fatales, ya que en esta gráfica no puedo sacar ninguna conclusion acerca de los accidentes fatales al estar tan desbalanceados los datos

In [ ]:
tiempo_colision_fatal = Datos_temporal.groupby('date')["fatal"].sum()
plot = tiempo_colision_fatal.plot(figsize = (15,5), title = "Deadly colissions over time")
plot.set_xlabel("Year")
plot.set_ylabel("Number of deadly colissions")

Voy a reducir el rango temporal para sacar conclusiones más especificas acerca de que época del año contribuye a que haya mas accidentes con fallecimientos. Partiendo de 2010

In [ ]:
df_reduced = Datos_temporal.loc[Datos_temporal["date"] >= "2010"]
tiempo_colision_fatal = df_reduced.groupby('date')["fatal"].sum()


plot = tiempo_colision_fatal.plot(figsize = (15,5), title = "Deadly colissions over time")
plot.set_xlabel("Year")
plot.set_ylabel("Number of deadly colissions")

In [ ]:
tiempo_colision_no_fatal = df_reduced.groupby('date')["non_fatal"].sum()

plot = tiempo_colision_no_fatal.plot(figsize = (15,5), title = "Deadly colissions over time", color="orange")
plot.set_xlabel("Year")
plot.set_ylabel("Number of deadly colissions")

Efectivamente existe una relacion temporal, a mediados de año es cuando mas accidentes 

Correlación de mi variable objetivo C_SEV

In [ ]:
from pandas.plotting import autocorrelation_plot
plt.figure(figsize=(15,5))
for c in tiempo_colisión.columns:
    autocorrelation_plot(tiempo_colisión[c][-60:],label=c);

La variable C_SEV se comporta igual para ambos valores

In [ ]:
plt.figure(figsize=(15,5))
sns.regplot('fatal', 'non_fatal', data=tiempo_colisión);

Ademas, aqui podemos ver que existe una relación lineal entre la fatalidad y la no fatalidad, propia del algoritmo regresión lineal 

In [ ]:
corr = tiempo_colisión.corr()
print("La correlación entre los accidentes fatales y no fatales es de", str(round(corr["fatal"]["non_fatal"], 3)))
corr

He decidido afrontar este apartado analizando aquellos factores que considero mas determinantes a la hora de que existan fallecimientos en un accidente, en vez de volver a implementar los algoritmos de clasificación y regresión.

## Análisis de la carretera.

Lo primero, defino las funciones que voy a aplicar en mis representaciones graficas; una función que mide la mortalidad de los accidentes(que sería una muestra más reducida ya que toma solo los fallecimientos), otra la mortalidad total (que irá siempre de la mano de la frecuencia ya que es un porcentaje y por tanto está influenciada por el desbalanceamiento de los datos), y por ultimo la frecuencia. Y también un boxplot

In [ ]:
### Función mortalidad (% / 100 Accidentes)
def mortalidad(columna, tipos, titulo, data):
    analysis = data.groupby([columna, "C_SEV"]).size()
    serie = list()
    for i in range(0, np.int8(len(analysis) / 2)):
        serie.append(analysis[:, 1].iloc[[i]].item() / (
                    analysis[:, 1].iloc[[i]].item() + (analysis[:, 2].iloc[[i]].item())) * 100)
    analysis = pd.Series(serie, index=range(0, np.int8(len(analysis) / 2)))
    plt.figure(figsize=(15, 5))
    plot = analysis.plot(kind="bar", title=titulo, color="#3A5683")
    plot.set_xticklabels(tipos, rotation=45)

In [ ]:
### Función mortalidad total
def mortalidadtotal(columna, tipos, titulo, data):
    analysis = data.groupby(columna)["C_SEV"].sum() / data["C_SEV"].sum() * 100
    plt.figure(figsize=(15, 5))
    plot = analysis.plot(kind="bar", title=titulo, color="#76B041")
    plot.set_xticklabels(tipos, rotation=45)

In [ ]:
### Función frecuencia absoluta
def frecuencia(columna, tipos, titulo, data):
    analysis = data.groupby(columna)["C_SEV"].count()
    plt.figure(figsize=(15, 5))
    plot = analysis.plot(kind="bar", title=titulo, color="#639A88")
    plot.set_xticklabels(tipos, rotation=45)

In [ ]:
def boxplot_fatality(var, data):
    f, ax = plt.subplots(figsize=(14, 6))
    sns.boxplot(x=var, y='C_SEV', data=data, orient='h')

In [ ]:
tipos = ['Mid-block','At an intersection','Intersection with parking lot entrance/exit',
            'Railroad crossing','Bridge','Tunnel','Passing or climbing lane',
             'Ramp','Traffic circle','Highway express lane', 'Other']

frecuencia("C_RCFG", tipos, "Collisions by road configuration", Datos)
mortalidadtotal("C_RCFG", tipos, "Percentage of deadly collisions by road configuration", Datos)
mortalidad("C_RCFG", tipos, "Mortality rate on accidents by road configuration", Datos)

Podemos concluir que hay un mayor indice de accidentes en el sentido de la marcha y en las intersecciones, aunque en relació´n con la mortalidad, lo que mas contribuye son los carriles VAO y los de adelantamiento. 

## Estado del asfalto

In [ ]:
tipos = ['Dry, normal','Wet','Snow','Slush ,wet snow','Icy','Sand/gravel/dirt','Muddy','Oil','Flooded', 'Other']
frecuencia("C_RSUR", tipos, "Collisions by surface",Datos)
mortalidadtotal("C_RSUR", tipos, "Percentage of deadly collisions by surface",Datos)
mortalidad("C_RSUR", tipos, "Mortality rate on accidents by surface",Datos)

En el caso del estado del asfalto, por temas de probabilidad, hay mas accidentes en condiciones normales, pero los accidentes mortales son caracterisitcos cuando el estado de la carretera es pésimo, con charcos, con grava...

## Condiciones climaticas

In [ ]:
tipos = ['Clear','Cloudy','Rain','Snow','Freezing Rain,Hail',
                'Low Visibility','Strong Wind','Other']
frecuencia("C_WTHR", tipos, "Collisions by weather",Datos)
mortalidadtotal("C_WTHR", tipos, "Percentage of deadly collisions by weather",Datos)
mortalidad("C_WTHR", tipos, "Mortality rate on accidents by weather",Datos)

Como las graficas anteriores, hay mas accidentes en condiciones climaticas normales, sin embargo, la tasa de mortalidad es mas alta cuando hay baja visibilidad, fuertes vientos o granizo

## Según tipo de vehiculo

In [ ]:
tipos = v_type = ['Light Duty', 'Cargo <4.5t', 'Truck <4.5t',
          'Truck >4.5t', 'Road tractor', 'School bus',
          'Small school bus', 'Urban bus', 'Motorcycle',
          'Off-road', 'Bicycle', 'Motorhome', 'Farm equip.',
          'Constru. equip.', 'Fire engine', 'Snowmobile',
          'Street car', 'Not vehicle', 'Others']
             
frecuencia("V_TYPE", tipos, "Collisions by vehicle type",Datos)
mortalidadtotal("V_TYPE", tipos, "Percentage of deadly collisions by vehicle type",Datos)
mortalidad("V_TYPE", tipos, "Mortality rate on accidents by vehicle type",Datos)

Aqui es mas de lo mismo, nos encontramos con que hay más accidentes con coches estandar, ya que por la cantidad es lo más probable. Aqui es interesante la tasa de mortalidad ya uqe se distribuye mas por los distintos valores. Se pueden sacar conclusiones respecto a que tipo de trabajo se realizan con cada tipo de vehiculo. Aunque hay que tener en cuenta que es posible que la muestra en este caso no sea representativa.



Ahora vamos a analizar que caracteristicas de los conductores contribuyen mas a que haya fallecimientos. En primer lugar, analicemos el sexo del conductor

In [ ]:
tipos = ["Female", "Male"]
frecuencia("P_SEX", tipos, "Collisions by sex", Datos)
mortalidadtotal("P_SEX", tipos, "Percentage of deadly collisions by sex", Datos)
mortalidad("P_SEX", tipos, "Mortality rate on accidents by sex", Datos)

Podemos ver una distribución balanceada. Puede ser porque hay más hombres en el mundo y además hay más hombres con carnet de conducir que mujeres. Además, en los trabajos de conducción suelen haber mas hombres que mujeres. Mueren más hombres que mujeres en los accidentes, puede ser por lo que acabo de mencionar y porque los hombres son más temerarios que las mujeres. Aqui podemos demostrar la falsedad del prejuicio de que las mujeres conducen peor.

## Según la edad del conductor

In [ ]:
tipos = sorted(np.int8(Datos['P_AGE'].dropna().unique()))
boxplot_fatality("P_AGE",Datos)
frecuencia("P_AGE", tipos, "Collisions by passenger age",Datos)
mortalidadtotal("P_AGE", tipos, "Percentage of deadly collisions by passenger age",Datos)
mortalidad("P_AGE", tipos, "Mortality rate on accidents by passenger age",Datos)

Aquellos conductores cuya edad oscila entre los 16 y 28 años, son mas propensos a tener accidentes. Sin embargo mueren los conductores de avanzada edad, con alguna pequeña excepción causada por la baja representatividad de la muestra.


## Según seguridad del conductor

In [ ]:
tipos = ['No safety device used','Safety device used','Helmet',
          'Reflective clothing',
          'Other safety device','No safety device equipped', 'Not applicable', 'Other']


frecuencia("P_SAFE", tipos, "Collisions by safety device used by passengers", Datos)
mortalidadtotal("P_SAFE", tipos, "Percentage of deadly collisions by safety device used by passengers", Datos)
mortalidad("P_SAFE", tipos, "Mortality rate on accidents by safety device used by passengers", Datos)

Este analisis es muy util para verificar la utilidad de los equipos de seguridad. En este caso, hay una baja representatividad debido a la muestra. Hay mas accidentes cuando se usa un equipo de seguridad, debido a que aumenta la muestra.

## Según si es motociclista, conductor, pasajero...

In [ ]:
tipos = ["Motor Vehicle Driver", "Motor Vehicle Passenger", "Pedestrian", "Bicyclist", "Motorcyclist"]
frecuencia("P_USER", tipos, "Collisions by passenger role",Datos)
mortalidadtotal("P_USER", tipos, "Percentage of deadly collisions by passenger role",Datos)
mortalidad("P_USER", tipos, "Mortality rate on accidents by passenger role",Datos)

Como era de esperar, mueren mas biciclistas ya que están mas expuestos al peligro. Y en el caso de la frecuencia de accidentes, vuelve a influir el peso de la muestra.

 # Dado un accidente, ¿se puede generar un modelo que prediga si habrá fallecimientos o no? ¿Si se va a necesitar tratamiento médico o no? Las aseguradoras tienen que inmovilizar capital para pagar estas casuísticas

**Este aparta y el siguiente no lo hemos podido comentar correctamente debido a que algunas lineas de codigo tardaban mas de una hora en ejecutarse**

In [ ]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

In [ ]:
Datos_test_dummy= Datos_test_dummy.drop(['P_ISEV','V_ID','P_ID', 'C_SEV'], axis= 1)

In [ ]:
Datos_test_dummy

Eliminamos estas 3 variables ya que P_ISEV da la misma informacion que C_SEV, y tanto V_ID y P_ID son el número de identidad de las personas involucradas que no nos dan ninguna info relevante

In [ ]:
Datos_train_dummy= Datos_train_dummy.drop(['P_ISEV','V_ID','P_ID'], axis= 1)

In [ ]:
Datos_train_dummy

In [ ]:
cat_cols = Datos_train_dummy.select_dtypes(include=['object', 'category']).columns.to_list()
numeric_cols = Datos_train_dummy.select_dtypes(include=['float64', 'int']).columns.to_list()

CReamos dos variables en las cuales ontenemos las variables numericas por un lado y por otro las variables no numericas (tanto "objecto" como "float"). De esta forma podremos trabajar mas adelante con ellas de manera mas comoda y diferencial


In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
                    [('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols)],
                    remainder='passthrough'
               )


Aprovechando las variables con solo las categoricas, realizamos un OHE para convertir en codigo binario las variables. Que dejen de ser categoricas. Esto nos hara mas facil trabajar con el dataset y es importante a la hora de realizar predicciones con arboles. 

In [ ]:
X_train_prep = preprocessor.fit_transform(Datos_train_dummy)
X_test_prep  = preprocessor.transform(Datos_test_dummy)

In [ ]:
X_train_prep = pd.DataFrame(X_train_prep)
X_test_prep  = pd.DataFrame(X_test_prep)

In [ ]:
X_train_prep

## GRID SEARCH BASADO EN OUT-BAG SCORE

In [ ]:
param_grid = ParameterGrid(
                {'n_estimators': [100],
                 'max_features': [10, 15, 19],
                 'max_depth'   : [5, 10, 15],
                 'criterion'   : ['gini', 'entropy']
                }
            )

# Loop para ajustar un modelo con cada combinación de hiperparámetros
# ==============================================================================
resultados = {'params': [], 'oob_accuracy': []}

for params in param_grid:
    
    modelo = RandomForestClassifier(
                oob_score    = True,
                n_jobs       = -1,
                random_state = 123,
                ** params
             )
    modelo.fit(X_train_prep, y_Datos_train)
    
    resultados['params'].append(params)
    resultados['oob_accuracy'].append(modelo.oob_score_)
    print(f"Modelo: {params} \u2713")



Modelo: {'criterion': 'gini', 'max_depth': 5, 'max_features': 10, 'n_estimators': 100} ✓


Con el grid search buscamos optimizar los hiperparametros, es decir, aquellos factores que definiran nuestros arboles. Estos factors son muy importantes y de ellos dependera la precision y eficiencia de los resultados finales. Por ejemplo, no es bueno que en un random forest los arboles o estvimadors que forman parte del metodo esten relacionados entre si. Si cogemos muchas variables para cada arbol es cada vez mas sencillo que haya arboles que cometan los mismos errores.Tampoco es bueno coger excesivamente pocas variables. Para cosas como esta nos sirve el GRID SEARCH. Para saber el nivel optimo de cada hiperparametro.

In [ ]:
 resultados = pd.DataFrame(resultados)
resultados = pd.concat([resultados, resultados['params'].apply(pd.Series)], axis=1)
resultados = resultados.drop(columns = 'params')
resultados = resultados.sort_values('oob_r2', ascending=False)
resultados.head(4)

#Libertad para generar análisis de valor y nuevas ideas. Se debe atacar mínimo un modelo (estimar si habrá fallecidos o no). Hecho esto, se puede plantear de forma opcional otros alcances (libertad para plantear opciones).

**Esteapartado no lo hemos podido comentar correctamente porque algunas lineas de codigo tardaban mas de una hora en ejecutarse**

## ARBOL DE DECISIÓN SIMPLE

In [ ]:
Datos_test_dummy= Datos_test_dummy.drop(['C_SEV'], axis= 1)

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
# Create Decision Tree classifer object
clf_tree = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf_tree = clf_tree.fit(Datos_train_dummy, y_Datos_train)
#Predict the response for test dataset
y_pred_tree=clf_tree.predict(Datos_test_dummy)

Los resultados de la matriz de confusion según el modelo de regresión logistica muestra que siendo la prediccion correcta habría 9826 colisiones sin fatalidades y en caso de que la predicción realizada no fuese correcta habria 823236 de colisiones sin fatalidad.

Por otro lado tanto si es correcta la predicción habría 4568
colisiones con al menos una fatalidad y si no es correcta habría 13860  colisiones con al menos una fatalidad.


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_Datos_test, y_pred_tree)
print(confusion_matrix)


Para valorar los resultados de la matrix anterior he utilizado distintas metricas
•	Precision: medimos la calidad. En este caso el 0,25% de las colisiones tendrían al menos una fatalidad y el modelo se equivocaría un 99,75 % de las veces cuando prediga que la colisión tiene al menos una fatalidad 
•	Recall: informa sobre la cantidad que el modelo es capaz de identificar. En este caso el modelo es capaz de identificar el 32% de las colisiones con al menos una fatalidad y al 98% de las colisiones con ninguna fatalidad
•	F1-Score:se utiliza para combinar las medidas de precisión y recall en un sólo valor.
•	Accuracy: mide el porcentaje de casos que el modelo ha acertado. Los resultados muestran que el modelo acierta un 97% de las veces


In [ ]:
print(classification_report(y_Datos_test,y_pred_tree))

## BAGGING CLASSIFIER

In [ ]:
escalar = StandardScaler()
Datos_train_dummy = escalar.fit_transform(Datos_train_dummy )
Datos_test_dummy = escalar.transform(Datos_test_dummy )

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.model_selection import StratifiedKFold,cross_validate

h = BaggingClassifier()
h.fit(Datos_train_dummy,y_Datos_train)
y_pred_bag= h.predict(Datos_test_dummy)

MATRIZ DE CONFUSIÓN

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_Datos_test, y_pred_bag)
print(confusion_matrix)

In [ ]:
print(classification_report(y_Datos_test,y_pred_bag))

## GRADIENT BOOSTING

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
modelo_gb = GradientBoostingClassifier(
              learning_rate=0.1,  max_depth=12,
              min_weight_fraction_leaf=0.02, n_estimators=130,
              random_state=2408,
              verbose=1)
modelo_gb.fit(Datos_test_dummy, y_Datos_test)
predicciones = modelo_gb.predict(Datos_test_dummy)

# Error de test del modelo inicial
rmse = mean_squared_error(
        y_true  = y_Datos_test,
        y_pred  = predicciones,
        squared = False
       )
print(f"El error (rmse) de test es: {rmse}")

In [ ]:
modelo_gb.score(Datos_train_dummy, y_Datos_test)

CONFUSION MATRIX

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_Datos_test, predicciones)
print(confusion_matrix)

In [ ]:
print(classification_report(y_Datos_test, predicciones))

In [ ]:
classifiers = [GradientBoostingClassifier()]# estudiar los hiperparametros y usar cualquier modelo aprendid

for classifier in classifiers:
  pipe = Pipeline(steps=[('classifier', classifier)])
  pipe.fit(Datos_train_dummy, Datos_test_dummy)   
  print(classifier)
  print("model score: %.3f" % pipe.score(Datos_train_dummy, Datos_test_dummy))

IMPORTANCIA VARIABLES

In [ ]:
feature_importance = modelo_gb.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
# plt.subplot(1, 2, 2)
plt.figure(figsize=(12, 20))
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, Datos_train_dummy.keys()[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

## REGRESION LOGISTICA

In [ ]:
from sklearn.linear_model import LogisticRegression
modelo_rl = LogisticRegression()
modelo_rl.fit(X_train_1s, y_train)
y_pred_lr=modelo_rl.predict(X_test_1)

CONFUSION MATRIX

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_lr)
print(confusion_matrix)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_lr))

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

logit_roc_auc = roc_auc_score(y_test, modelo_rl.predict(X_test_1))
fpr, tpr, thresholds = roc_curve(y_test, modelo_rl.predict_proba(X_test_1)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()